In [3]:
# [+] SparkSession 설정
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('udf').getOrCreate()

In [4]:
# 샘플 데이터: 한신대 맛집 및 대표메뉴
hsu_restaurants = [
    ('진현가든', '삼치돌솥밥', '경기 오산시 양산로 340딩', 3.5),
    ('대광생막창', '돼지막창', '경기 화성시 한신대길 99', 4.0),
    ('찌개동아리', '제육전골', '경기 오산시 한신대133번길 4', 3.5),
    ('한판삼겹', '항정살', '경기 오산시 양산로410번길 8', 3.5),
    ('화락', '초밥', '경기 오산시 양산로 347 대성빌딩1층', 3.5),
    ('해우리', '해물라면', '경기 오산시 한신대길 135 1층', 4.5),
    ('행복한콩박사', '맑은순두부', '경기 오산시 양산로398번길 8-11', 3.5)
]


In [5]:
# [+] 스키마 정의
schema = ['restaurant_name', 'speciality', 'address', 'score']

In [6]:
# [+] 데이터프레임 생성
df = spark.createDataFrame(data = hsu_restaurants, schema = schema)

In [7]:
# [+] 데이터프레임 출력
df.show()

+---------------+----------+------------------------------+-----+
|restaurant_name|speciality|                       address|score|
+---------------+----------+------------------------------+-----+
|       진현가든|삼치돌솥밥|      경기 오산시 양산로 340딩|  3.5|
|     대광생막창|  돼지막창|       경기 화성시 한신대길 99|  4.0|
|     찌개동아리|  제육전골|   경기 오산시 한신대133번길 4|  3.5|
|       한판삼겹|    항정살|   경기 오산시 양산로410번길 8|  3.5|
|           화락|      초밥|경기 오산시 양산로 347 대성...|  3.5|
|         해우리|  해물라면|  경기 오산시 한신대길 135 1층|  4.5|
|   행복한콩박사|맑은순두부|경기 오산시 양산로398번길 8-11|  3.5|
+---------------+----------+------------------------------+-----+



In [8]:
# [+] 데이터프레임 스키마 출력
df.printSchema()

root
 |-- restaurant_name: string (nullable = true)
 |-- speciality: string (nullable = true)
 |-- address: string (nullable = true)
 |-- score: double (nullable = true)



In [9]:
# [+] Temporary View 생성
df.createOrReplaceTempView('restaurants')

### User Defined Function 생성
1. translate(): Google Translation API를 이용하여 한글 식당 이름을 영문으로 번역하는 함수
2. scale_score(): 5점 만점 평점을 100점 스케일로 변환하는 함수

In [10]:
# Google translation 라이브러리 설치
!pip install googletrans==4.0.0-rc1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.
jupyterlab-server 2.8.2 requires jupyter-server~=1.4, but you have jupyter-server 2.5.0 which is incompatible.


  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17415 sha256=9ea3d975ef96bdeccbf7ae3fe7d5418135ecb9d1cc37c0e27c7732474fe8d235
  Stored in directory: c:\users\hnw\appdata\local\pip\cache\wheels\60\b3\27\d8aff3e2d5c2d0d97a117cdf0d5f13cd121e2c2b5fb49b55a0
Successfully built googletrans
  Attempting uninstall: idna
    Found existing installation: idna 3.2
    Uninstalling idna-3.2:
      Successfully uninstalled idna-3.2
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0


cookiecutter 1.7.2 requires Jinja2<3.0.0, but you have jinja2 3.1.2 which is incompatible.
cookiecutter 1.7.2 requires MarkupSafe<2.0.0, but you have markupsafe 2.1.2 which is incompatible.


In [13]:
# Google translator 임포트
from googletrans import Translator

translator = Translator()

In [14]:
# 번역 테스트
                                   # source  destination
result = translator.translate("찌개 동아리", src='ko', dest='en')
print(result)

Translated(src=ko, dest=en, text=Stew, pronunciation=None, extra_data="{'confiden...")


In [23]:
# [+] 번역 결과 출력
result.text

'Stew'

In [16]:
# [+] extra_data 출력
result.extra_data

{'confidence': None,
 'parts': [<googletrans.models.TranslatedPart at 0x205f8f14970>],
 'origin_pronunciation': 'jjigae dong-ali',
 'parsed': [['jjigae dong-ali',
   None,
   None,
   [[[0, [[[None, 6]], [True]]]], 6],
   [['찌개 동아리', None, None, 6]],
   None,
   ['찌개 동아리', 'ko', 'en', True]],
  [[[None,
     None,
     None,
     None,
     None,
     [['Stew',
       None,
       None,
       None,
       [['Stew', [5], []], ['Stew club', [11], []]]]]]],
   'en',
   1,
   'ko',
   ['찌개 동아리', 'ko', 'en', True]],
  'ko']}

In [17]:
# 영어 발음(pronunciation) 출력
result.extra_data.get('origin_pronunciation')

'jjigae dong-ali'

In [36]:
# UDF 1: 한글->영문 번역 함수
    # 함수 생성 필요
def translate(text):
    from googletrans import Translator
    translator = Translator()
    result = translator.translate(text, src='ko', dest='en')
    return result.extra_data.get('origin_pronunciation')

# UDF 등록
spark.udf.register('translate', translate)

<function __main__.translate(text)>

In [26]:
df.show()

+---------------+----------+------------------------------+-----+
|restaurant_name|speciality|                       address|score|
+---------------+----------+------------------------------+-----+
|       진현가든|삼치돌솥밥|      경기 오산시 양산로 340딩|  3.5|
|     대광생막창|  돼지막창|       경기 화성시 한신대길 99|  4.0|
|     찌개동아리|  제육전골|   경기 오산시 한신대133번길 4|  3.5|
|       한판삼겹|    항정살|   경기 오산시 양산로410번길 8|  3.5|
|           화락|      초밥|경기 오산시 양산로 347 대성...|  3.5|
|         해우리|  해물라면|  경기 오산시 한신대길 135 1층|  4.5|
|   행복한콩박사|맑은순두부|경기 오산시 양산로398번길 8-11|  3.5|
+---------------+----------+------------------------------+-----+



In [37]:
# SQL문 처리
spark.sql("SELECT restaurant_name, translate(restaurant_name) AS restaurant_name_en \
            FROM restaurants").show()

+---------------+--------------------+
|restaurant_name|  restaurant_name_en|
+---------------+--------------------+
|       진현가든|      jinhyeongadeun|
|     대광생막창|daegwangsaengmagc...|
|     찌개동아리|      jjigaedong-ali|
|       한판삼겹|      hanpansamgyeob|
|           화락|              hwalag|
|         해우리|              haeuli|
|   행복한콩박사|haengboghankongbagsa|
+---------------+--------------------+



In [28]:
# Annotation 방식으로 UDF 등록하기
from pyspark.sql.functions import udf

In [31]:
# UDF 2: score -> 100점 스케일 변환 함수
# @udf...... udf로 바로 등록
    # @udf('int')
    # def sclae_score(score):
    #     return score * 20

def scale_score(score):
    return score * 20

In [32]:
# udf 등록
spark.udf.register("scale_score", scale_score)

<function __main__.scale_score(score)>

In [38]:
# SQL문처리
spark.sql("SELECT restaurant_name, translate(restaurant_name) AS restaurant_name_en, \
            speciality, address, scale_score(score) AS score_scaled \
            FROM restaurants").show()

+---------------+--------------------+----------+------------------------------+------------+
|restaurant_name|  restaurant_name_en|speciality|                       address|score_scaled|
+---------------+--------------------+----------+------------------------------+------------+
|       진현가든|      jinhyeongadeun|삼치돌솥밥|      경기 오산시 양산로 340딩|        70.0|
|     대광생막창|daegwangsaengmagc...|  돼지막창|       경기 화성시 한신대길 99|        80.0|
|     찌개동아리|      jjigaedong-ali|  제육전골|   경기 오산시 한신대133번길 4|        70.0|
|       한판삼겹|      hanpansamgyeob|    항정살|   경기 오산시 양산로410번길 8|        70.0|
|           화락|              hwalag|      초밥|경기 오산시 양산로 347 대성...|        70.0|
|         해우리|              haeuli|  해물라면|  경기 오산시 한신대길 135 1층|        90.0|
|   행복한콩박사|haengboghankongbagsa|맑은순두부|경기 오산시 양산로398번길 8-11|        70.0|
+---------------+--------------------+----------+------------------------------+------------+

